In [1]:
import os
os.chdir('/Users/benjaminshih/opt/miniconda3/envs/filer/lib/python3.8/site-packages')
%matplotlib inline
%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2
import requests
import matplotlib as mpl
import pandas as pd
from matplotlib import rc
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import  html
import plotly.graph_objects as go

#aesthetic params
nord = ['8fbcbb', '81a1c1', 'bf616a', 'd08770', 'ebcb8b', 'a3be8c', 'b48ead', 'd8dee9']
nord_hex = ["#"+i for i in nord]
custom_colors = mpl.cycler('color', nord)
mpl.rcParams['axes.prop_cycle'] = custom_colors
mpl.rcParams['figure.dpi'] = 300

rc('font',**{'family':'serif','serif':['Times']})

In [2]:
#global vars
FILER_URL= 'https://tf.lisanwanglab.org/FILER'

data_region = 'get_data_region.php' 
metadata = 'get_metadata.php'
overlap_track = 'get_overlapping_tracks_by_coord.php'

In [3]:
def request_data(base_url: str, endpoint: str, params: dict) -> "JSON":
    """
    Returns the JSON data for a particular endpoint (one of track data, metadata or overlap data) and 
    particular parameters

    Parameters:
        base_url (str): FILER or GADB base URL to fetch from
        endpoint (str): endpoint to fetch from, either data region, metadata, or overlapping tracks
        params (dict): request-specific parameters
        
    Returns:
        JSON dictionary containing API call data
    """
    r = requests.get(base_url + "/" + endpoint, params=params)
    data = r.json()
    
    if r.status_code != requests.codes.ok:
        print("Request failed")
        return
    
    return data

### Metadata Example

Generate HTML Heatmap with hover information from FILER API dataframe:
- Demonstrates ability to quickly query and return visualizations of data within FILER database

In [4]:
data = request_data(FILER_URL, metadata, 
                    {'genomeBuild':'hg19', 'dataSource':'ROADMAP', 'assayType':'ChIP-seq'})
df = pd.DataFrame(data)
df = df[["cell type", "system category", "Number of intervals"]]

fig = px.density_heatmap(df, x="system category", y="cell type", marginal_x="histogram", marginal_y="histogram", color_continuous_scale=nord_hex)
fig.write_html("test.html", auto_open=True)

In [ ]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
data = request_data(FILER_URL, overlap_track, {'genomeBuild':'hg19', 'region':'chr1:1103243-1103243', 
                                               'filterString':'.', 'outputFormat':'json'})
df = pd.DataFrame(data)
df = df[df.columns.difference(["wget command", "tabix_index Download", "Link out URL", "File name", "Raw File URL", "Raw file download", "Raw file md5", "Processed file md5", \
                               "Processed File Download URL", "filepath", "File format", "File size", "Downloaded date", "Date added to GADB", "Release date"])]
df = df[["Data Category", "system category", "bp covered"]]
px.sunburst(df, path=["system category"], values=["Data Category"])
fig.write_html("test2.html", auto_open=True)

In [ ]:
data = request_data(FILER_URL, overlap_track, {'genomeBuild':'hg19', 'region':'chr1:1103243-1103243', 
                                               'filterString':'.', 'outputFormat':'json'})
df = pd.DataFrame(data)
df = df[df.columns.difference(["wget command", "tabix_index Download", "Link out URL", "File name", "Raw File URL", "Raw file download", "Raw file md5", "Processed file md5", \
                               "Processed File Download URL", "filepath", "File format", "File size", "Downloaded date", "Date added to GADB", "Release date"])]
df = df[["Data Category", "system category", "bp covered"]]
fig = px.histogram(df, x="Data Category", y="bp covered", 
                   marginal="box", # or violin, rug
                   hover_data=['system category'])
fig.write_html("test3.html", auto_open=True)

#Error in ```dash``` module for interactivity (dropdowns, etc.), will debug env

In [ ]:
fig = go.Figure(
    data=[go.Bar(y=[2, 1, 3])],
    layout_title_text="Native Plotly rendering in Dash"
)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph", figure=fig),
])

app.run_server(mode='jupyterlab')

In [ ]:
%tb